In [59]:
!pip install mip

In [60]:
from queue import Queue

In [61]:
from mip import *

In [108]:
def read(filename):
    with open(filename, 'r') as file:
        lista = file.readlines()
        lista = [x.strip().split() for x in lista]
        lista = [[int(y) for y in x] for x in lista]
        variaveis = lista[0][0]
        num_restricoes = lista[0][1]
        objetivo = lista[1]
        restricoes = lista[2:]
    return variaveis, num_restricoes, objetivo, restricoes

def solve_relaxation(model, restricoes, objetivo, variaveis):
    model.objective = xsum(objetivo[i] * model.vars[i] for i in range(variaveis))

    # Adiciona as restrições ao modelo
    for data in restricoes:
        model += xsum(data[i] * model.vars[i] for i in range(variaveis)) <= data[-1]

    model.optimize()

    if model.num_solutions:
        return model.objective_value, [v.x for v in model.vars]
    else:
        return None, None

#Determina a variável de ramificação
def branch(solucao, alpha=0.0001):
    nao_binarias = [(i, abs(x - round(x))) for i, x in enumerate(solucao) if abs(x - round(x)) > alpha]
    if not nao_binarias:
        return None

    nao_binarias.sort(key=lambda x: -abs(x[1] - 0.5))
    branchV = nao_binarias[0][0]
    return branchV

def branch_and_bound(filename):
    variaveis, num_restricoes, objetivo, restricoes = read(filename)

    # Cria uma fila
    queue = Queue()
    queue.put(([], {i: None for i in range(variaveis)}))

    best_solution = None
    best_objective = float('-inf')

    while not queue.empty():
        restricao_atual, integridade_atual = queue.get()

        m = Model(sense=MAXIMIZE)

        m.vars = [m.add_var(var_type=CONTINUOUS, lb=0, ub=1) for i in range(variaveis)]

        for (x, y) in restricao_atual:
            if y is not None:
                m += m.vars[x] == y

        # Resolve a relaxação linear do modelo atual
        objective_value, solution = solve_relaxation(m, restricoes, objetivo, variaveis)

        # Se o valor objetivo da relaxação linear é pior que o melhor encontrado, ignora o ramo
        if objective_value is None or objective_value <= best_objective:
            continue

        # Se a solução é viável, atualiza a melhor solução
        if all(x in (0, 1) for x in solution):
            if objective_value > best_objective:
                best_solution = solution
                best_objective = objective_value
            continue

        # Determina a próxima variável para ramificação
        branch_var = branch(solution)
        if branch_var is not None:
            # Ramifica nas variáveis, adicionando restrições e colocando novos nós na fila
            for i in [0, 1]:
                nova_restricao = restricao_atual + [(branch_var, i)]
                queue.put((nova_restricao, integridade_atual))

    return best_objective, best_solution

In [110]:
optimal_value, optimal_solucao = branch_and_bound('teste1.txt')
print("Optimal value:", optimal_value)
print("Optimal solucao:", optimal_solucao)

Optimal value: 20.0
Optimal solucao: [0.0, 0.0, 0.0, 0.0, 1.0, 1.0, 0.0]


In [111]:
optimal_value, optimal_solucao = branch_and_bound('teste2.txt')
print("Optimal value:", optimal_value)
print("Optimal solucao:", optimal_solucao)

Optimal value: 24.0
Optimal solucao: [0.0, 0.0, 0.0, 0.0, 0.0, 1.0, 1.0, 0.0, 1.0]


In [112]:
optimal_value, optimal_solucao = branch_and_bound('teste3.txt')
print("Optimal value:", optimal_value)
print("Optimal solucao:", optimal_solucao)

Optimal value: 19.0
Optimal solucao: [0.0, 0.0, 1.0, 0.0, 0.0, 0.0, 1.0, 0.0, 0.0]


In [113]:
optimal_value, optimal_solucao = branch_and_bound('teste4.txt')
print("Optimal value:", optimal_value)
print("Optimal solucao:", optimal_solucao)

Optimal value: 10.0
Optimal solucao: [0.0, 0.0, 1.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0]
